# Avaliando O nível do Cantareira em anos de seca com NDWI com [Landsat-7](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T1_L2?hl=pt-br)

- Foi usado como exemplo o ano `úmido de 2011` e `seco de 2015` e centrado no Cantareira, especificamente nos rios Jaguari e Jacarei.





# Inicializando o GEE

In [ ]:
# importando o GEE e Geemap
import ee
import geemap

# inicializando GEE
geemap.ee_initialize(project='ee-enrique')

# Definindo região

In [ ]:
# região de interesse
regiao = ee.Geometry.Rectangle([-46.4728, -23.0389, -46.2757, -22.8967])

# criando um mapa interativo
Map = geemap.Map()

# centraliza o mapa
Map.centerObject(regiao, 12)

# adicionando basemap
Map.add_basemap('SATELLITE')

# contorno da região
Map.addLayer(regiao, {'color': 'yellow', 'fillColor': '00000000', 'opacity': 0.3}, 'Região')

# exibe na tela
Map

In [ ]:
# extraindo as coordenadas de latitude/longitude que limita a região do mapa selecionada
#regiao = Map.draw_last_feature.geometry()

# mostra as latitudes/longitudes limitantes da região
#regiao

# Mapa Cantareira sem Zoom com combinação de RGB e Falsa cor para destacar água
- Dados usados: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T1_L2?hl=pt-br
- Anos: 2010 (úmido) x 2014 (seco).
- Jan 2011 (úmido) x jan 2015 (seco)

In [ ]:
# região de interesse
regiao = ee.Geometry.Rectangle([-46.4728-0.5, -23.0389, -46.2757+0.5, -22.8967])

# aplica fator de escala
def fator_escala(img):

    # Aplica fatores de escala
    opticas = img.select('SR_B.*').multiply(2.75e-05).add(-0.2)
    termais = img.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Retorna imagem com bandas escaladas
    return (img.addBands(opticas, None, True)
               .addBands(termais, None, True)
               .copyProperties(img, ['system:time_start'])
               .set('date', img.date().format('YYYY-MM-dd')))

# 2011 - CHEIA
imagens_cheia = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
                  .filterBounds(regiao) \
                  .filterDate('2011-01-01','2012-01-01') \
                  .filter(ee.Filter.lt('CLOUD_COVER', 5)) \
                  .map(fator_escala)

# 2015 - SECA
imagens_seca = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
                 .filterBounds(regiao) \
                 .filterDate('2015-01-01','2016-01-01') \
                 .filter(ee.Filter.lt('CLOUD_COVER', 5)) \
                 .map(fator_escala)

# parâmetros de visualização
param_vis_rgb = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 0.03, 'max': 0.19}
param_vis_agua = {'bands': ['SR_B7', 'SR_B4', 'SR_B3'], 'min': 0.03, 'max': 0.19}

# monta painel com 4 mapas
geemap.linked_maps(rows = 2,
                   cols = 2,
                   height = "200px",
                   center = [-22.97, -46.39],
                   zoom = 10.5,

                   ee_objects = [imagens_cheia.mean().clip(regiao),
                                 imagens_seca.mean().clip(regiao),

                                 imagens_cheia.mean().clip(regiao),
                                 imagens_seca.mean().clip(regiao)],

                   vis_params = [param_vis_rgb,
                                 param_vis_rgb,

                                 param_vis_agua,
                                 param_vis_agua],

                   labels = ['CHEIA (2011): RGB',
                             'SECA (2015): RGB',
                             'CHEIA (2011): FALSA COR',
                             'SECA (2015): FALSA COR'],

                   label_position = "topright")

In [ ]:
# bandas
imagens_cheia.first().bandNames().getInfo()

In [ ]:
# mostra datas das imagens
imagens_cheia

In [ ]:
# mostra datas das imagens
imagens_seca

# Mapa Cantareira com Zoom e calculo da àrea antes com CHEIA (2011) e depois SECA (2015)

- Anos: 2010 (úmido) x 2014 (seco).
- Jan 2011 (úmido) x jan 2015 (seco)

In [ ]:
#===============================================================================================#
#                                     REGIÃO DE INTERESSE
#===============================================================================================#
# região de interesse
regiao = ee.Geometry.Rectangle([-46.4728, -23.0389, -46.2757, -22.8967])

#===============================================================================================#
#                                     CARREGA OS DADOS
#===============================================================================================#
# 2011 - CHEIA
imagens_cheia = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
                  .filterBounds(regiao) \
                  .filterDate('2011-01-01','2012-01-01') \
                  .filter(ee.Filter.lt('CLOUD_COVER', 5))

# 2015 - SECA
imagens_seca = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
                 .filterBounds(regiao) \
                 .filterDate('2015-01-01','2016-01-01') \
                 .filter(ee.Filter.lt('CLOUD_COVER', 5))

#===============================================================================================#
#                                     CALCULA NDWI
#===============================================================================================#
# calcula NDWI
def ndwi(img):

    # fator de escala das bandas óticas
    bandas_oticas = img.select('SR_B.*').multiply(2.75e-05).add(-0.2)

    # fator de escala de bandas termais
    bandas_termais = img.select('ST_B.*').multiply(0.00341802).add(149.0)

    # NDWI
    ndwi_image = img.normalizedDifference(['SR_B2', 'SR_B4']).rename('ndwi')

    # aplicando e adicionando as bandas com fator de escala
    return img.addBands(bandas_oticas, None, True) \
              .addBands(bandas_termais, None, True) \
              .addBands(ndwi_image) \
              .copyProperties(img, ['system:time_start']) \
              .set('date', img.date().format('YYYY-MM-dd'))

# aplica a função de NDWI em cada imagem
imagens_cheia_ndwi = imagens_cheia.map(ndwi)
imagens_seca_ndwi = imagens_seca.map(ndwi)

# cria as máscaras para identificar valores máximos de NDWI
mask_agua_cheia = imagens_cheia_ndwi.select('ndwi').max().gt(0).selfMask().rename('cheia').set('date', '2019-06-01')
mask_agua_seca = imagens_seca_ndwi.select('ndwi').max().gt(0).selfMask().rename('seca').set('date', '2021-06-01')

#===============================================================================================#
#                               CALCULA ÁREA COM ÁGUA
#===============================================================================================#
# junta as imagens da CHEIA e SECA
mask_agua_cheia_seca = mask_agua_cheia.clip(regiao).addBands(mask_agua_seca.clip(regiao))

# cálcula a área de cada pixel em km²:
# 1. Multiplica pela área do pixel em m²
# 2. Divide por 1.000.000 para converter para km²
imagem_area_km2 = mask_agua_cheia_seca.multiply(ee.Image.pixelArea()).divide(1000000)

# acumula todos os pixels que estão em km²
soma_area_km2 = imagem_area_km2.reduceRegion(reducer=ee.Reducer.sum(),
                                             geometry=regiao,
                                             scale=30,
                                             maxPixels=1e13,
                                             bestEffort=True)

# obtém os valores em km²
area_cheia_km2 = soma_area_km2.get('cheia').getInfo()
area_seca_km2 = soma_area_km2.get('seca').getInfo()

print(f'Área período de CHEIA: {area_cheia_km2:.1f} km²')
print(f'Área período de SECA: {area_seca_km2:.1f} km²')
dif = area_cheia_km2 - area_seca_km2
print(f'DIMINUIÇÃO da área: {dif:.1f} km²')

#===============================================================================================#
#                           PLOTA MAPA DE RGB E MASCARA DE ÁGUA
#===============================================================================================#
# parâmetros de visualização
param_vis_rgb = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 0.03, 'max': 0.19}
param_vis_agua = {'bands': ['SR_B7', 'SR_B4', 'SR_B3'], 'min': 0.03, 'max': 0.19}
param_vis_mask = {'palette' : ['blue'], 'min': 0, 'max': 1}

# monta painel com 4 mapas
geemap.linked_maps(rows = 1,
                   cols = 2,
                   height = "200px",
                   center = [-22.97, -46.39],
                   zoom = 10.5,

                   ee_objects = [mask_agua_cheia.clip(regiao),
                                 mask_agua_seca.clip(regiao)],

                   vis_params = [param_vis_mask,
                                 param_vis_mask],

                   labels = ['CHEIA (2011)', 'SECA (2015)'],

                   label_position = "topright")